In [7]:
import folium
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Sample code
# Coordinates for Bangladesh
bd_coordinates = (23.685, 90.356)

# Create a map centered around Bangladesh with OpenStreetMap tileset
bd_map = folium.Map(location=bd_coordinates, zoom_start=7, tiles='OpenStreetMap')

# Custom icon for the bridges
icon = folium.Icon(color="blue", icon="info-sign")

# Coordinates of important bridges in Bangladesh
bridges = {
    "Padma Bridge": (23.8251, 90.3686),
    "Bangabandhu Bridge": (24.0794, 89.6529),
    "Jamuna Bridge": (24.4366, 89.8930),
    "Meghna Bridge": (23.0238, 90.7287)
}

# Add markers for each bridge with customized icons
for bridge, coords in bridges.items():
    folium.Marker(location=coords, popup=bridge, icon=icon).add_to(bd_map)

# Display the map
bd_map



To Do

@Lukas
>add additional datafiles to git
>Code uit assignment 3 invoegen invoegen
>dataframe met benodigde wegen maken
> try to merge rmms data with bmms

@timon en @fred
>htm bestanden uit datafolder inlezen
>data uit htm bestand scrapen
>pd.read_htm , 5th element is the table
> read in all vehicles types
> split vehicles into cargo and non-cargo


@bram if lukas not succesfull try via aggregating
>combine bridge df and traffic flow df (via chainage)



Hoe moet het er uiteindelijk uit zien?
>criticality: simple traffic flow (broken down into Heavy Truck, Medium Truck, Small Truck)
> vulnerability: quality col from bmms df + bridge length
> Quality ordinal variable, bridge length


Outputs/ plot:
>choropleths of traffic flow data
> 


First, let us define a function to find the RMMS folder location.

In [8]:
def get_rmms_relative_path():
    # Get the current working directory
    current_directory = os.getcwd()

    # Navigate one folder up
    parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))

    # Go inside the "data" folder
    data_folder = os.path.join(parent_directory, 'data')

    # Go inside the "RMMS" folder
    rmms_folder = os.path.join(data_folder, 'RMMS')

    # Print the relative path
    relative_path = os.path.relpath(rmms_folder, start=current_directory)
    
    return relative_path


In [9]:
# Example usage:
relative_path_to_rmms = get_rmms_relative_path()
print(relative_path_to_rmms)

..\data\RMMS
